<a href="https://colab.research.google.com/github/kushal-g/FlightDelayPredictor/blob/master/Models/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
from multiprocessing import Pool
from xgboost import XGBClassifier

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd /content/gdrive/My\ Drive

/content/gdrive/My Drive


In [0]:
df = pd.read_csv("flights_logistic_final.csv",dtype={"ORIGIN_AIRPORT":object,"DESTINATION_AIRPORT":object})

In [0]:
df.head()

,Unnamed: 0,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,ARRIVAL_DELAY,DIVERTED,CANCELLED,SCHEDULED_DEPARTURE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY_BOOL,TAIL_NUMBERS_BINNED,ORIGIN_AIRPORT_BINNED,DESTINATION_AIRPORT_BINNED,MONTH_2,MONTH_3,MONTH_4,MONTH_5,MONTH_6,MONTH_7,MONTH_8,MONTH_9,MONTH_10,MONTH_11,MONTH_12,DAY_OF_WEEK_2,...,DESTINATION_AIRPORT_20,DESTINATION_AIRPORT_21,DESTINATION_AIRPORT_22,DESTINATION_AIRPORT_23,DESTINATION_AIRPORT_24,"SCHEDULED_DEPARTURE_(1000.0, 1200.0]","SCHEDULED_DEPARTURE_(1200.0, 1400.0]","SCHEDULED_DEPARTURE_(1400.0, 1600.0]","SCHEDULED_DEPARTURE_(1600.0, 1800.0]","SCHEDULED_DEPARTURE_(1800.0, 2000.0]","SCHEDULED_DEPARTURE_(200.0, 400.0]","SCHEDULED_DEPARTURE_(2000.0, 2200.0]","SCHEDULED_DEPARTURE_(2200.0, 2400.0]","SCHEDULED_DEPARTURE_(400.0, 600.0]","SCHEDULED_DEPARTURE_(600.0, 800.0]","SCHEDULED_DEPARTURE_(800.0, 1000.0]","SCHEDULED_ARRIVAL_(1000.0, 1200.0]","SCHEDULED_ARRIVAL_(1200.0, 1400.0]","SCHEDULED_ARRIVAL_(1400.0, 1600.0]","SCHEDULED_ARRIVAL_(1600.0, 1800.0]","SCHEDULED_ARRIVAL_(1800.0, 2000.0]","SCHEDULED_ARRIVAL_(200.0, 400.0]","SCHEDULED_ARRIVAL_(2000.0, 2200.0]","SCHEDULED_ARRIVAL_(2200.0, 2400.0]","SCHEDULED_ARRIVAL_(400.0, 600.0]","SCHEDULED_ARRIVAL_(600.0, 800.0]","SCHEDULED_ARRIVAL_(800.0, 1000.0]",AIRLINE_AS,AIRLINE_B6,AIRLINE_DL,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN
0,0,2015,12,25,5,EV,3846,N21154,HOB,IAH,-5.0,6.0,901.0,101.0,86.0,74.0,501,1115.0,6.0,-20.0,0,0,"(800.0, 1000.0]","(1000.0, 1200.0]",0,29,15,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,1,2015,1,1,4,EV,5690,N836AS,IAD,PIT,-4.0,16.0,1258.0,61.0,57.0,37.0,182,1335.0,4.0,-8.0,0,0,"(1200.0, 1400.0]","(1200.0, 1400.0]",0,40,1,2,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,2,2015,4,26,7,AA,2200,N3BWAA,DFW,FLL,8.0,38.0,2051.0,168.0,182.0,136.0,1119,7.0,8.0,22.0,0,0,"(2000.0, 2200.0]","(2200.0, 2400.0]",1,28,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,2015,4,27,1,DL,1672,N992DL,ATL,BUF,-2.0,28.0,846.0,119.0,129.0,96.0,712,1022.0,5.0,8.0,0,0,"(800.0, 1000.0]","(1000.0, 1200.0]",1,26,0,2,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,4,2015,10,20,2,NK,326,N633NK,12892,12266,-5.0,10.0,1610.0,190.0,205.0,166.0,1379,2056.0,29.0,10.0,0,0,"(1600.0, 1800.0]","(2000.0, 2200.0]",1,4,2,3,0,0,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [0]:
for columns in X.columns:
  print(columns)

Unnamed: 0
ELAPSED_TIME
DISTANCE
DIVERTED
CANCELLED
MONTH_2
MONTH_3
MONTH_4
MONTH_5
MONTH_6
MONTH_7
MONTH_8
MONTH_9
MONTH_10
MONTH_11
MONTH_12
DAY_OF_WEEK_2
DAY_OF_WEEK_3
DAY_OF_WEEK_4
DAY_OF_WEEK_5
DAY_OF_WEEK_6
DAY_OF_WEEK_7
TAIL_NUMBER_1
TAIL_NUMBER_2
TAIL_NUMBER_3
TAIL_NUMBER_4
TAIL_NUMBER_5
TAIL_NUMBER_6
TAIL_NUMBER_7
TAIL_NUMBER_8
TAIL_NUMBER_9
TAIL_NUMBER_10
TAIL_NUMBER_11
TAIL_NUMBER_12
TAIL_NUMBER_13
TAIL_NUMBER_14
TAIL_NUMBER_15
TAIL_NUMBER_16
TAIL_NUMBER_17
TAIL_NUMBER_18
TAIL_NUMBER_19
TAIL_NUMBER_20
TAIL_NUMBER_21
TAIL_NUMBER_22
TAIL_NUMBER_23
TAIL_NUMBER_24
TAIL_NUMBER_25
TAIL_NUMBER_26
TAIL_NUMBER_27
TAIL_NUMBER_28
TAIL_NUMBER_29
TAIL_NUMBER_30
TAIL_NUMBER_31
TAIL_NUMBER_32
TAIL_NUMBER_33
TAIL_NUMBER_34
TAIL_NUMBER_35
TAIL_NUMBER_36
TAIL_NUMBER_37
TAIL_NUMBER_38
TAIL_NUMBER_39
TAIL_NUMBER_40
TAIL_NUMBER_41
TAIL_NUMBER_42
TAIL_NUMBER_43
TAIL_NUMBER_44
TAIL_NUMBER_45
TAIL_NUMBER_46
ORIGIN_AIRPORT_1
ORIGIN_AIRPORT_2
ORIGIN_AIRPORT_3
ORIGIN_AIRPORT_4
ORIGIN_AIRPORT_5
ORIGIN_

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X = df.drop(["YEAR","MONTH","DAY","DAY_OF_WEEK","AIRLINE","FLIGHT_NUMBER","TAIL_NUMBER","ORIGIN_AIRPORT","DESTINATION_AIRPORT","DEPARTURE_DELAY","TAXI_OUT","WHEELS_OFF","SCHEDULED_TIME","AIR_TIME","WHEELS_ON","TAXI_IN","ARRIVAL_DELAY","ARRIVAL_DELAY_BOOL","SCHEDULED_DEPARTURE","SCHEDULED_ARRIVAL","TAIL_NUMBERS_BINNED","ORIGIN_AIRPORT_BINNED","DESTINATION_AIRPORT_BINNED"],axis=1)

In [0]:
y = df["ARRIVAL_DELAY_BOOL"]

In [0]:
X_train,X_test,y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [0]:
model = XGBClassifier(n_estimators=500, random_state=42)

In [0]:
eval_set = [(X_train, y_train), (X_test, y_test)]
eval_metric = ["auc","error"]

In [0]:
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)

In [0]:
X_test.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_test.columns.values]

In [0]:
import time
%time model.fit(X_train, y_train)

CPU times: user 9min 50s, sys: 519 ms, total: 9min 51s
Wall time: 9min 51s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
pred_test = model.predict(X_test)

In [0]:
from sklearn.metrics import confusion_matrix

In [0]:
matrix = confusion_matrix(y_test,pred_test)

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
rfc = RandomForestClassifier(n_estimators=500)

In [0]:
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
rfc_pred = rfc.predict(X_test)

In [0]:
matrix = confusion_matrix(y_test,rfc_pred)

In [0]:
((matrix[0][0]+matrix[1][1])/(matrix[0][0]+matrix[0][1]+matrix[1][0]+matrix[1][1]))*100

71.953